Caching

Saves consult results for future reuse improving the performance
Reduces the API calls


In [10]:
from langchain_openai import OpenAI
from langchain_community.cache import InMemoryCache, SQLiteCache
from langchain_core.globals import set_llm_cache
import json
import hashlib
import os
import yaml

In [11]:
with open("config.yaml", "r") as config_file:
    config = yaml.safe_load(config_file)
    os.environ["OPENAI_API_KEY"] = config["OPENAI_API_KEY"]

In [12]:
openai = OpenAI(model_name='gpt-3.5-turbo-instruct')
#define to use in-memory cache
set_llm_cache(InMemoryCache())

In [13]:
prompt = "Tell me a joke about programming."
response1 = openai.invoke(prompt)
print("First response: (API call made): ", response1)

First response: (API call made):  

Why do programmers prefer dark mode?

Because light attracts bugs!


In [14]:
response2 = openai.invoke(prompt)
print("Second response: (From cache, no API call): ", response2)

Second response: (From cache, no API call):  

Why do programmers prefer dark mode?

Because light attracts bugs!


Disc/Database

In [16]:
set_llm_cache(SQLiteCache(database_path="openai_cache.db"))

In [17]:
prompt = "Tell me a joke about programming."

In [18]:
response1 = openai.invoke(prompt)
print("First response: (API call made): ", response1)

First response: (API call made):  

Why do programmers prefer dark mode?

Because light attracts bugs!


In [19]:
response2 = openai.invoke(prompt)
print("Second response: (From cache, no API call): ", response2)

Second response: (From cache, no API call):  

Why do programmers prefer dark mode?

Because light attracts bugs!


Caching Personalizado

In [20]:
class SimpleDiskCache:
    def __init__(self, cache_dir='cache_dir'):
        self.cache_dir = cache_dir
        os.makedirs(self.cache_dir, exist_ok=True)

    def _get_cache_path(self, key):
        hashed_key = hashlib.md5(key.encode()).hexdigest() #hasg cria nome de arquivo único
        return os.path.join(self.cache_dir, f"{hashed_key}.json")

    def lookup(self, key, llm_string):
        cache_path = self._get_cache_path(key)
        if os.path.exists(cache_path):
            with open(cache_path, 'r') as f:
                return json.load(f)
        return None

    def update(self, key, value, llm_string):
        cache_path = self._get_cache_path(key)
        with open(cache_path, 'w') as f:
            json.dump(value, f)

In [21]:
cache = SimpleDiskCache()
set_llm_cache(cache)
prompt = 'Tell me about the World War One.'

In [22]:
def invoke_with_cache(llm, prompt, cache):
    cached_response = cache.lookup(prompt, "")
    if cached_response:
        print("Using cache:")
        return cached_response

    response = llm.invoke(prompt)
    cache.update(prompt, response, "")
    return response

In [23]:
response1 = invoke_with_cache(openai, prompt, cache)
response_text1 = response1.replace("\n", " "
                                   
                                   )
print("First response: (API call made): ", response_text1)

First response: (API call made):    World War One, also known as the First World War or the Great War, was a global conflict that lasted from 1914 to 1918. It involved many of the world's major powers, divided into two opposing alliances: the Allies (led by France, the United Kingdom, and Russia) and the Central Powers (led by Germany, Austria-Hungary, and the Ottoman Empire).  The war was sparked by the assassination of Archduke Franz Ferdinand of Austria-Hungary in June 1914, which led to a chain of events that eventually resulted in the outbreak of war. The conflict was primarily fought in Europe, but also saw battles and campaigns in Africa, Asia, and the Middle East.  The war was characterized by trench warfare, in which soldiers dug complex systems of trenches to protect themselves from enemy fire. This led to a stalemate on the Western Front, with neither side able to make significant advances for several years.  The war saw the use of new and advanced technologies, such as mach

In [24]:
response2 = invoke_with_cache(openai, prompt, cache)
response_text2 = response2.replace("\n", " "
                                   
                                   )
print("First response: (API call made): ", response_text2)

Using cache:
First response: (API call made):    World War One, also known as the First World War or the Great War, was a global conflict that lasted from 1914 to 1918. It involved many of the world's major powers, divided into two opposing alliances: the Allies (led by France, the United Kingdom, and Russia) and the Central Powers (led by Germany, Austria-Hungary, and the Ottoman Empire).  The war was sparked by the assassination of Archduke Franz Ferdinand of Austria-Hungary in June 1914, which led to a chain of events that eventually resulted in the outbreak of war. The conflict was primarily fought in Europe, but also saw battles and campaigns in Africa, Asia, and the Middle East.  The war was characterized by trench warfare, in which soldiers dug complex systems of trenches to protect themselves from enemy fire. This led to a stalemate on the Western Front, with neither side able to make significant advances for several years.  The war saw the use of new and advanced technologies,